# Imports

In [ ]:
%cd ..
%cd location_controller/location_controller/

In [ ]:

import numpy as np
import open3d as o3d

import rosbag2_py
import rclpy.serialization
from sensor_msgs.msg import PointCloud2
from utils import pointcloud2_to_open3d

# Extract Map

## Change working directory for the Notebook to read the recordings

In [ ]:
%cd ../../
%cd recordings/

## Save the voxelized map

In [ ]:
def read_bag(base_path):
       storage_options = rosbag2_py.StorageOptions(uri=base_path, storage_id="sqlite3")
       converter_options = rosbag2_py.ConverterOptions(input_serialization_format = "cdr", output_serialization_format = "cdr")
       reader = rosbag2_py.SequentialReader()
       reader.open(storage_options, converter_options)
       return reader

In [ ]:
original_map = []

bag_original = read_bag("map/original_map/original_map_0.db3")

while bag_original.has_next():
    
    # Read the next message in the queue
    topic, msg, t = bag_original.read_next()

    # Deserialize the msg to extract the x, y and z values
    original_map = rclpy.serialization.deserialize_message(msg, PointCloud2)

In [ ]:
original_map = pointcloud2_to_open3d(original_map)

# Métricas y visualización

## Métricas

### Distancia Nube-a-Nube

In [ ]:
def cloud_to_cloud_distance(pc1, pc2):
    # pc1 y pc2 son open3d.geometry.PointCloud
    dist1 = np.asarray(pc1.compute_point_cloud_distance(pc2))
    dist2 = np.asarray(pc2.compute_point_cloud_distance(pc1))
    return {
        "mean_dist": (np.mean(dist1) + np.mean(dist2)) / 2,
        "max_dist": max(np.max(dist1), np.max(dist2)),
        "hausdorff": max(np.max(dist1), np.max(dist2))
    }

In [ ]:
print(cloud_to_cloud_distance(original_map, original_map))